# **Librerias usadas**

In [327]:
import pandas as pd
import numpy as np
from meteostat import Stations, Daily
from datetime import datetime
import csv
import os
import requests
import re
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import folium

# **Preparación de la información**

### **1. Fase de preparación: Generación del catálogo**

In [196]:
# Datos extraídos previamente de SENAMHI (snapshot)
DATA_RAW = [
    "AMAZONAS_EL PINTOR", "AMAZONAS_MAGUNCHAL", "ANCASH_OCROS", "ANCASH_RECUAY", 
    "ANCASH_MAYORARCA", "ANCASH_SIHUAS", "ANCASH_BUENA VISTA", "ANCASH_PIRA", 
    "ANCASH_CHACCHAN", "ANCASH_MILPO", "APURIMAC_CURAHUASI", "AREQUIPA_MADRIGAL", 
    "AREQUIPA_IMATA", "AREQUIPA_PILLONES", "AREQUIPA_SIBAYO", "AREQUIPA_YANAQUIHUA", 
    "AREQUIPA_PULLHUAY (AYAHUASI)", "AREQUIPA_COTAHUASI", "AREQUIPA_PAMPA BLANCA", 
    "AREQUIPA_SALAMANCA", "AREQUIPA_MACHAHUAY", "AREQUIPA_CHOCO", "AREQUIPA_CHACHAS", 
    "AREQUIPA_AYO", "AREQUIPA_ANDAHUA", "AREQUIPA_APLAO", "AREQUIPA_PUNTA ATICO", 
    "AREQUIPA_CARAVELI", "AREQUIPA_LA JOYA", "AREQUIPA_PAMPA DE MAJES", "AREQUIPA_HUAMBO", 
    "AREQUIPA_LA ANGOSTURA", "AREQUIPA_CHIVAY", "AREQUIPA_ORCOPAMPA", "AREQUIPA_EL FRAYLE", 
    "AREQUIPA_CHIGUATA", "AREQUIPA_LAS SALINAS", "AREQUIPA_LA PAMPILLA", "AREQUIPA_CHICHAS", 
    "AREQUIPA_PORPERA", "AYACUCHO_PUQUIO", "AYACUCHO_CORACORA", "AYACUCHO_LAMPA", 
    "AYACUCHO_VILCASHUAMAN", "AYACUCHO_CHILCAYOC", "AYACUCHO_PAUZA", "AYACUCHO_LUCANAS", 
    "AYACUCHO_HUAC-HUAS", "AYACUCHO_LLAUTA", "CAJAMARCA_CUTERVO", "CAJAMARCA_QUEROCOTILLO", 
    "CAJAMARCA_QUEBRADA SHUGAR", "CAJAMARCA_CHUGUR", "CAJAMARCA_LA CASCARILLA", 
    "CAJAMARCA_CHONTALI", "CAJAMARCA_EL LIMON", "CAJAMARCA_HACIENDA PUCARA", 
    "CAJAMARCA_SALLIQUE", "CAJAMARCA_TOCMOCHE", "CAJAMARCA_LLAMA", "CAJAMARCA_CHOTANO LAJAS", 
    "CAJAMARCA_COCHABAMBA", "CAJAMARCA_CACHACHI", "CAJAMARCA_CAJABAMBA", "CAJAMARCA_SAN JUAN", 
    "CAJAMARCA_ASUNCION", "CAJAMARCA_AUGUSTO WEBERBAUER", "CAJAMARCA_SAN BENITO", 
    "CAJAMARCA_CHILETE", "CAJAMARCA_CHIRINOS", "CAJAMARCA_TABACONAS", "CAJAMARCA_SAN MARCOS", 
    "CAJAMARCA_LIVES", "CAJAMARCA_UDIMA", "CAJAMARCA_CHANCAY BAÑOS", "CUSCO_GRANJA KCAYRA", 
    "CUSCO_ACOMAYO", "CUSCO_POMACANCHI", "CUSCO_PISAC", "CUSCO_PARURO", "CUSCO_CAY CAY", 
    "CUSCO_COLQUEPATA", "CUSCO_CCATCCA", "HUANCAVELICA_HUANCALPI", "HUANCAVELICA_ACOBAMBA", 
    "HUANCAVELICA_LIRCAY", "HUANCAVELICA_HUACHOS", "HUANCAVELICA_SAN JUAN DE CASTROVIRREYNA", 
    "HUANCAVELICA_CHOCLOCOCHA", "HUANCAVELICA_CORDOVA", "HUANCAVELICA_TUNEL CERO", 
    "HUANCAVELICA_SANTIAGO DE CHOCORVOS", "HUANCAVELICA_TAMBO", "HUANCAVELICA_ACOSTAMBO", 
    "HUANCAVELICA_SALCABAMBA", "HUANUCO_CANCHAN", "HUANUCO_HUANUCO", "HUANUCO_SAN RAFAEL", 
    "HUANUCO_TINGO MARIA", "HUANUCO_TULUMAYO", "ICA_SAN CAMILO", "ICA_HACIENDA BERNALES", 
    "ICA_PAMPA BLANCA", "ICA_RIO GRANDE", "ICA_OCUCAJE", "ICA_HUAMANI", "ICA_FONAGRO (CHINCHA)", 
    "ICA_SAN JUAN DE YANAC", "ICA_COPARA", "JUNIN_SANTA ANA", "JUNIN_VIQUES", "JUNIN_INGENIO", 
    "JUNIN_RICRAN", "JUNIN_CARHUACAYAN", "JUNIN_HUAYAO", "JUNIN_YANTAC", "JUNIN_JAUJA", 
    "LA LIBERTAD_JULCAN", "LA LIBERTAD_CALLANCAS", "LA LIBERTAD_SINSICAP", 
    "LA LIBERTAD_HUAMACHUCO", "LA LIBERTAD_CACHICADAN", "LA LIBERTAD_HUACAMARCANGA", 
    "LA LIBERTAD_QUIRUVILCA", "LA LIBERTAD_MOLLEPATA", "LA LIBERTAD_HUANGACOCHA", 
    "LA LIBERTAD_TALLA (GUADALUPE)", "LAMBAYEQUE_REQUE", "LAMBAYEQUE_CAYALTI", 
    "LAMBAYEQUE_CUEVA BLANCA", "LAMBAYEQUE_JAYANCA (LA VIÑA)", "LAMBAYEQUE_VIRREY", 
    "LAMBAYEQUE_LAMBAYEQUE", "LAMBAYEQUE_PUCHACA", "LAMBAYEQUE_EL ESPINAL", "LIMA_GORGOR", 
    "LIMA_CANTA", "LIMA_ARAHUAY", "LIMA_ÑAÑA", "LIMA_CHOSICA", "LIMA_HUARANGAL", 
    "LIMA_ANDAJES", "LIMA_OYON", "LIMA_SANTA ROSA", "LIMA_PARQUIN", "LIMA_PICOY", 
    "LIMA_PACCHO", "LIMA_PACHAMACHAY", "LIMA_ALCANTARILLA", "LIMA_RIO BLANCO", "LIMA_TANTA", 
    "LIMA_SAN PEDRO DE PILAS", "LIMA_HUAÑEC", "LIMA_VILCA", "LIMA_CARANIA", "LIMA_AYAVIRI", 
    "LIMA_YAURICOCHA", "LIMA_SHEQUE", "LIMA_CARAMPOMA", "LIMA_ANTIOQUIA", "LIMA_MATUCANA", 
    "LIMA_SANTA CRUZ", "LIMA_HUAMANTANGA", "LIMA_HUAROS", "LIMA_PARIACANCHA", "LIMA_PAMPA LIBRE", 
    "LIMA_LOMAS DE LACHAY", "LIMA_SANTIAGO DE TUNA", "LIMA_SANTA EULALIA", "LIMA_CANCHACALLA", 
    "LIMA_HUANCATA", "LIMA_AUTISHA", "LIMA_SAN LAZARO DE ESCOMARCA", "LIMA_LANGA", 
    "LIMA_LACHAQUI", "LIMA_CAÑETE", "LIMA_PACARAN", "LIMA_HUAYAN", "LIMA_DONOSO", "LIMA_PIRCA", 
    "LIMA_PALLAC", "LORETO_PUNCHANA", "LORETO_MANITI", "LORETO_TIMICURILLO", 
    "LORETO_FRANCISCO ORELLANA", "LORETO_BELLAVISTA", "LORETO_LA LIBERTAD", "LORETO_SANTA CRUZ", 
    "LORETO_SANTA MARIA DE NANAY", "LORETO_MAZAN", "LORETO_SHANUSI", "LORETO_SAN RAMON", 
    "LORETO_BAGAZAN", "LORETO_SANTA RITA DE CASTILLA", "LORETO_SANTA ROSA", "LORETO_BRETAÑA", 
    "LORETO_GENARO HERRERA", "LORETO_JUANCITO", "MOQUEGUA_MOQUEGUA", "MOQUEGUA_QUINISTAQUILLAS", 
    "MOQUEGUA_CALACOA", "MOQUEGUA_ICHUÑA", "MOQUEGUA_UBINAS", "MOQUEGUA_PUNTA COLES", 
    "MOQUEGUA_ILO", "PIURA_MIRAFLORES", "PIURA_SAN MIGUEL", "PIURA_AYABACA", 
    "PIURA_SAUSAL DE CULUCAN", "PIURA_SAPILLICA", "PIURA_PORCULLA", "PIURA_HUARMACA", 
    "PIURA_HACIENDA SHUMAYA", "PIURA_SONDORILLO", "PIURA_TULUCE", "PIURA_SAN PEDRO", 
    "PIURA_CHULUCANAS", "PIURA_MORROPON", "PIURA_HACIENDA BIGOTE", "PIURA_LA ESPERANZA", 
    "PIURA_LANCONES", "PIURA_MALLARES", "PIURA_PANANGA", "PIURA_CHUSIS", "PIURA_BERNAL", 
    "PUNO_PUNO", "PUNO_ISLA TAQUILE", "PUNO_CAPACHICA", "PUNO_AZANGARO", "PUNO_ARAPA", 
    "PUNO_PROGRESO", "PUNO_MUÑANI", "PUNO_CRUCERO", "PUNO_JULI", "PUNO_DESAGUADERO", 
    "PUNO_PIZACOMA", "PUNO_ILAVE", "PUNO_CAPAZO", "PUNO_MAZO CRUZ", "PUNO_HUANCANE", 
    "PUNO_COJATA", "PUNO_LAMPA", "PUNO_PAMPAHUTA", "PUNO_PUCARA", "PUNO_CRUCERO ALTO", 
    "PUNO_AYAVIRI", "PUNO_SANTA ROSA", "PUNO_CHUQUIBAMBILLA", "PUNO_HUARAYA MOHO", "PUNO_PUTINA", 
    "PUNO_ANANEA", "PUNO_CABANILLAS", "PUNO_CUYO CUYO", "PUNO_TAHUACO - YUNGUYO", "PUNO_ISLA SUANA", 
    "TACNA_JORGE BASADRE", "TACNA_CALANA", "TACNA_SAMA GRANDE", "TACNA_TOQUELA", "TACNA_PALCA", 
    "TACNA_PAUCARANI", "TACNA_LOCUMBA", "TACNA_CHUAPALCA", "TACNA_TARATA", "TACNA_TALABAYA", 
    "TACNA_SITAJARA", "TACNA_SUSAPAYA", "TACNA_CHALLAPALCA", "TUMBES_PUERTO PIZARRO", 
    "TUMBES_EL TIGRE", "TUMBES_EL SALTO", "TUMBES_MATAPALO", "UCAYALI_EL MARONAL"
]

def generar_csv_estaciones():
    print("Generando archivo 'lista_estaciones_senamhi.csv'...")
    
    data_procesada = []
    for item in DATA_RAW:
        partes = item.split('_', 1)
        if len(partes) == 2:
            data_procesada.append({
                "departamento": partes[0],
                "estacion": partes[1],
                "codigo_completo": item
            })
            
    with open('lista_estaciones_senamhi.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["departamento", "estacion", "codigo_completo"])
        writer.writeheader()
        writer.writerows(data_procesada)
        
    print(f"¡Listo! {len(data_procesada)} estaciones exportadas.")

if __name__ == "__main__":
    generar_csv_estaciones()

Generando archivo 'lista_estaciones_senamhi.csv'...
¡Listo! 273 estaciones exportadas.


### **2. Fase de análisis: Detección de nieve**

In [198]:
def analizar_nieve_local(ruta_archivo):
    print(f"--- Procesando archivo: {ruta_archivo} ---")
    
    try:
        # El SENAMHI suele entregar archivos de texto separados por tabulaciones, comas o espacios.
        # Este comando intenta detectar automáticamente la estructura.
        # 'sep=None' usa el motor de python para detectar el separador.
        # 'skiprows' salta las primeras lineas si tienen metadatos (ajustar si es necesario)
        df = pd.read_csv(ruta_archivo, sep=None, engine='python')
        
        # Limpieza de nombres de columnas (quita espacios extra)
        df.columns = [c.strip().upper() for c in df.columns]
        print("Columnas encontradas:", df.columns.tolist())

        # Buscamos columnas clave (T.Minima y Precipitacion)
        # Nombres comunes en SENAMHI: 'MIN', 'TMIN', 'PRECIP', 'PP'
        col_min = next((c for c in df.columns if c in ['MIN', 'TMIN', 'TEMP MIN', 'T.MIN']), None)
        col_precip = next((c for c in df.columns if c in ['PRECIP', 'PP', 'LLUVIA', 'PLUVIOMETRIA']), None)
        col_fecha = next((c for c in df.columns if c in ['FECHA', 'DIA', 'DATE']), None)

        if not col_min or not col_precip:
            print("⚠️ Error: No encontré las columnas de Temperatura Mínima o Precipitación.")
            print("Por favor abre el archivo y dime cómo se llaman los encabezados.")
            return

        # Convertir datos a números (los errores 'S/D' se vuelven NaN)
        df[col_min] = pd.to_numeric(df[col_min], errors='coerce')
        df[col_precip] = pd.to_numeric(df[col_precip], errors='coerce')

        # === LÓGICA DE NIEVE ===
        # Regla: Si llovió (Precip > 0) Y la temperatura mínima fue muy baja (<= 2°C)
        def detectar_fenomeno(row):
            t = row[col_min]
            p = row[col_precip]
            
            if pd.isna(t) or pd.isna(p): return "Sin Datos"
            
            if p > 0:
                if t <= 1.0: return "❄️ NIEVE (Alta prob.)"
                elif t <= 3.0: return "💧 AGUANIEVE"
                else: return "🌧️ Lluvia líquida"
            return "☁️ Seco"

        df['CONDICION'] = df.apply(detectar_fenomeno, axis=1)

        # Filtramos solo los días de interés
        nieve = df[df['CONDICION'].str.contains("NIEVE|AGUANIEVE")]

        if not nieve.empty:
            print(f"\n✅ ¡Se encontraron {len(nieve)} eventos de nieve/aguanieve!")
            cols_mostrar = [col for col in [col_fecha, col_min, col_precip, 'CONDICION'] if col]
            print(nieve[cols_mostrar].to_string(index=False))
            
            # Guardar reporte
            nieve.to_csv("reporte_nieve.csv", index=False)
            print("\nReporte guardado en 'reporte_nieve.csv'")
        else:
            print("\nNo se detectaron condiciones de nieve en este archivo.")

    except Exception as e:
        print(f"Error leyendo el archivo: {e}")

# --- EJECUCIÓN ---
# Asegúrate de poner el nombre correcto de tu archivo descargado aquí:
archivo_descargado = 'datos_senamhi.txt' 

# Crea un archivo dummy de prueba si no existe, para que veas que funciona
if not os.path.exists(archivo_descargado):
    with open(archivo_descargado, 'w') as f:
        f.write("FECHA,TMIN,PP\n2023-01-01,5.0,0\n2023-06-15,-2.0,5.0\n2023-07-20,0.5,2.0\n")
    print("(Creé un archivo de prueba automático para la demostración)")

analizar_nieve_local(archivo_descargado)

--- Procesando archivo: datos_senamhi.txt ---
Columnas encontradas: ['FECHA', 'TMIN', 'PP']

✅ ¡Se encontraron 1 eventos de nieve/aguanieve!
     FECHA  TMIN  PP             CONDICION
2023-06-20  -1.0 5.0 ❄️ NIEVE (Alta prob.)

Reporte guardado en 'reporte_nieve.csv'


### **3. Fase de simulación: Generador de escenarios**

In [200]:
nombre_archivo = 'datos_senamhi.txt'

# Sobrescribimos el archivo con datos mixtos (Lluvia, Nieve y Seco)
with open(nombre_archivo, 'w') as f:
    f.write("FECHA,TMIN,PP\n")
    f.write("2023-01-05,12.0,0.0\n")   # Seco
    f.write("2023-02-14,14.5,15.0\n")  # ✅ Lluvia Moderada (T=14°C)
    f.write("2023-02-15,13.0,2.5\n")   # ✅ Lluvia Ligera (T=13°C)
    f.write("2023-06-20,-1.0,5.0\n")   # ❄️ Nieve (Se ignorará)
    f.write("2023-03-10,18.0,45.0\n")  # ✅ Lluvia Fuerte (T=18°C)

print(f"Archivo '{nombre_archivo}' actualizado con datos de lluvia.")

Archivo 'datos_senamhi.txt' actualizado con datos de lluvia.


### **4. Fase de integración: Clasificador final**

In [202]:
# 1. CREAMOS EL ARCHIVO CON DATOS VARIADOS
datos = [
    ["2023-01-05",  15.0,  10.0], # Lluvia
    ["2023-02-14",  12.5,  25.0], # Lluvia fuerte
    ["2023-06-15",  -3.0,   5.0], # Nieve
    ["2023-06-20",   0.5,   8.0], # Nieve
    ["2023-07-01",  -5.0,   0.0], # Seco
    ["2023-09-10",   8.0,   2.0], # Lluvia debil
    ["2023-12-24",   1.8,  12.0]  # Nieve/Aguanieve
]

df = pd.DataFrame(datos, columns=["FECHA", "TMIN", "PP"])

# 2. CLASIFICAMOS CADA DÍA
def definir_clima(fila):
    temp = fila["TMIN"]
    lluvia = fila["PP"]
    
    if lluvia > 0:
        if temp <= 2.0: # Umbral de frío
            return "❄️ NIEVE"
        else:
            return "🌧️ LLUVIA"
    else:
        return "☁️ SECO"

df["RESULTADO"] = df.apply(definir_clima, axis=1)

# 3. MOSTRAMOS LA LISTA FINAL
print("--- LISTA CLASIFICADA ---")
print(df.to_string(index=False))

# (Opcional) Guardar en Excel/CSV
df.to_csv("reporte_final_clasificado.csv", index=False)

--- LISTA CLASIFICADA ---
     FECHA  TMIN   PP RESULTADO
2023-01-05  15.0 10.0 🌧️ LLUVIA
2023-02-14  12.5 25.0 🌧️ LLUVIA
2023-06-15  -3.0  5.0  ❄️ NIEVE
2023-06-20   0.5  8.0  ❄️ NIEVE
2023-07-01  -5.0  0.0   ☁️ SECO
2023-09-10   8.0  2.0 🌧️ LLUVIA
2023-12-24   1.8 12.0  ❄️ NIEVE


# **Web scraping**

## **Sección 1: Inteligencia de red y extracción**

In [308]:
def ver_tripas_del_archivo():
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/js/mapa-estaciones-v.1.0.0.js"
    
    print(f"👀 Inspeccionando: {url}")
    try:
        response = requests.get(url)
        texto = response.text
        
        print("\n" + "="*20 + " CONTENIDO DEL ARCHIVO (PRIMEROS 1000 CARACTERES) " + "="*20)
        # Imprimimos solo el inicio para no saturar tu pantalla, pero suficiente para ver el patrón
        print(texto[:1000]) 
        print("="*60)
        
    except Exception as e:
        print(f"Error: {e}")

ver_tripas_del_archivo()

👀 Inspeccionando: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/js/mapa-estaciones-v.1.0.0.js

==================== CONTENIDO DEL ARCHIVO (PRIMEROS 1000 CARACTERES) ====================
var mymap;
const CENTERLAT = -9.1951786;
const CENTERLONG = -74.9904165;
const ICONANCHOR = [9, 27];
const POPUPANCHOR = [0, -30];
const MAPIMG = 'images/grilla-peru07.png';
const IMAGEBOUNDS = [[-19.044705, -81.92], [0.335, -68.074]];

createMap();

function fillIcon_2(tipo, estado) {

    if (tipo == 'M' && estado == 'REAL') {
        var ico = 'ico_meteo.svg';

    } else if (tipo == 'H' && estado == 'REAL') {
        var ico = 'ico_hidro.svg';

    } else if (tipo == 'M' && estado == 'DIFERIDO') {
        var ico = 'ico_meteo.svg';

    } else if (tipo == 'H' && estado == 'DIFERIDO') {
        var ico = 'ico_hidro.svg';

    } else if (tipo == 'M' && estado == 'AUTOMATICA') {
        var ico = 'ico_meteoBk.svg';

    } else if (tipo == 'H' && estado == 'AUTOMATICA') {
        var ico = 'ico_hid

In [312]:
def escaneo_profundo():
    print("🕵️ INICIANDO ESCANEO PROFUNDO DE DATOS...")
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    # 1. Definimos los dos lugares donde pueden esconderse los datos
    fuentes = [
        ("ARCHIVO JS", "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/js/mapa-estaciones-v.1.0.0.js"),
        ("PÁGINA PRINCIPAL", "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/")
    ]
    
    encontrado = False

    for nombre, url in fuentes:
        print(f"\n--- Analizando: {nombre} ---")
        try:
            response = requests.get(url, headers=headers, timeout=10)
            contenido = response.text
            
            # Buscamos cualquier patrón que parezca una coordenada o una estación
            # Buscamos algo como: lat : -12.34  o  "lat": -12.34
            # Capturamos 50 caracteres antes y después para ver el contexto
            patron_busqueda = r'.{0,50}(lat|lon)["\s:]+[-0-9.]{3,10}.{0,50}'
            
            matches = re.findall(patron_busqueda, contenido)
            
            if matches:
                print(f"✅ ¡EUREKA! Datos encontrados en {nombre}.")
                print(f"📊 Cantidad de posibles puntos: {len(matches)}")
                print("\nMUESTRA DEL FORMATO REAL (Copia esto):")
                print("-" * 60)
                # Mostramos los primeros 3 ejemplos para entender el formato
                for i in range(min(3, len(matches))):
                    print(f"Ejemplo {i+1}: ... {matches[i]} ...")
                print("-" * 60)
                encontrado = True
                break # Si lo encontramos, dejamos de buscar
            else:
                print(f"❌ No hay rastro de datos en {nombre}.")
                
        except Exception as e:
            print(f"⚠️ Error leyendo {nombre}: {e}")

    if not encontrado:
        print("\n😱 MALAS NOTICIAS: Los datos no están ni en el JS ni en el HTML.")
        print("Es probable que se carguen desde un archivo .json oculto que Selenium no capturó.")

escaneo_profundo()

🕵️ INICIANDO ESCANEO PROFUNDO DE DATOS...

--- Analizando: ARCHIVO JS ---
❌ No hay rastro de datos en ARCHIVO JS.

--- Analizando: PÁGINA PRINCIPAL ---
✅ ¡EUREKA! Datos encontrados en PÁGINA PRINCIPAL.
📊 Cantidad de posibles puntos: 970

MUESTRA DEL FORMATO REAL (Copia esto):
------------------------------------------------------------
Ejemplo 1: ... lat ...
Ejemplo 2: ... lat ...
Ejemplo 3: ... lat ...
------------------------------------------------------------


In [314]:
import requests
import re
import pandas as pd

def extraer_desde_html():
    print("🌐 Conectando con SENAMHI (Página Principal)...")
    
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        contenido = response.text
        
        # --- PASO 1: DIAGNÓSTICO VISUAL (INFALIBLE) ---
        # Buscamos la posición de la palabra "lat" y cortamos el texto alrededor.
        # Esto NO usa regex, así que no puede fallar al mostrarte los datos.
        indice = contenido.find('lat"') # Buscamos con comilla primero
        if indice == -1: 
            indice = contenido.find('lat') # Si falla, buscamos sin comilla
            
        print("\n" + "="*50)
        print("🔎 VISOR DE RAYOS X (Así se ven los datos reales):")
        print("="*50)
        # Mostramos 100 caracteres antes y después de encontrar "lat"
        print(contenido[indice-50 : indice+100])
        print("="*50 + "\n")

        # --- PASO 2: EXTRACCIÓN AUTOMÁTICA ---
        print("⛏️ Intentando extracción masiva...")
        
        # PATRÓN HÍBRIDO: 
        # Busca algo como: "nom": "LIMA", ... "lat": -12.04, "lon": -77.02
        # Funciona con o sin espacios extra.
        regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
        
        matches = re.findall(regex, contenido)
        
        if len(matches) > 0:
            print(f"✅ ¡ÉXITO! Se detectaron {len(matches)} estaciones.")
            
            datos = []
            for nombre, lat, lon in matches:
                datos.append({
                    "ESTACION": nombre.strip(),
                    "LATITUD": float(lat),
                    "LONGITUD": float(lon),
                    "FUENTE": "HTML_EMBEDDED"
                })
            
            # Guardar CSV
            df = pd.DataFrame(datos)
            df.to_csv("SENAMHI_ESTACIONES_FINAL.csv", index=False, encoding='utf-8-sig')
            print("📁 Archivo guardado: SENAMHI_ESTACIONES_FINAL.csv")
            print(df.head())
            
        else:
            print("⚠️ El patrón estándar falló. Revisa el texto en el 'VISOR DE RAYOS X' arriba.")
            print("Es posible que usen comillas simples (') en lugar de dobles (\") o que el orden sea diferente.")

    except Exception as e:
        print(f"❌ Error crítico: {e}")

if __name__ == "__main__":
    extraer_desde_html()

🌐 Conectando con SENAMHI (Página Principal)...

🔎 VISOR DE RAYOS X (Así se ven los datos reales):
= [
        {"nom": "MONTEGRANDE", "cate": "CO", "lat": -7.22499, "lon": -79.15323, "ico": "M", "cod": "100090","cod_old": "000396", "estado": "DIFERI

⛏️ Intentando extracción masiva...
✅ ¡ÉXITO! Se detectaron 969 estaciones.
📁 Archivo guardado: SENAMHI_ESTACIONES_FINAL.csv
              ESTACION  LATITUD  LONGITUD         FUENTE
0          MONTEGRANDE -7.22499 -79.15323  HTML_EMBEDDED
1           YONAN GORE -7.25592 -79.09908  HTML_EMBEDDED
2         QUEROCOTILLO -6.27366 -79.03696  HTML_EMBEDDED
3  PUENTE  MAYGASBAMBA -6.67411 -78.52437  HTML_EMBEDDED
4      QUEBRADA SHUGAR -6.68778 -78.45694  HTML_EMBEDDED


## **Sección 2: El buscador de vecinos**

In [316]:
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="SENAMHI_ESTACIONES_FINAL.csv"):
    """
    Toma una coordenada (de Visual Crossing/OpenSky) y encuentra la estación SENAMHI más cercana.
    """
    # 1. Cargar la base de datos que acabamos de crear
    try:
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        return "Error: No se encuentra el archivo CSV de estaciones."

    # 2. Calcular la distancia a CADA estación (Fórmula de distancia simple)
    # Distancia = Raíz((Lat1-Lat2)² + (Lon1-Lon2)²)
    df['distancia'] = np.sqrt(
        (df['LATITUD'] - lat_objetivo)**2 + 
        (df['LONGITUD'] - lon_objetivo)**2
    )

    # 3. Ordenar y tomar la ganadora (la de menor distancia)
    estacion_ganadora = df.sort_values('distancia').iloc[0]

    return {
        "ESTACION": estacion_ganadora['ESTACION'],
        "LATITUD": estacion_ganadora['LATITUD'],
        "LONGITUD": estacion_ganadora['LONGITUD'],
        "DISTANCIA_APROX": round(estacion_ganadora['distancia'] * 111, 2) # *111 para convertir grados a km aprox
    }

# --- PRUEBA DEL FLUJO (SIMULACIÓN) ---
print("--- 🔄 SIMULANDO PASE DE BATUTA ENTRE ETAPAS ---")

# 1. Output de ETAPA 1 (Visual Crossing) para "Lima"
lat_vc = -12.02
lon_vc = -77.11
print(f"📍 Dato recibido de Etapa 1: {lat_vc}, {lon_vc}")

# 2. Ejecución de ETAPA 3 (Búsqueda SENAMHI)
resultado = encontrar_estacion_cercana(lat_vc, lon_vc)

print("\n🎯 RESULTADO ETAPA 3:")
print(f"   La estación SENAMHI más cercana es: {resultado['ESTACION']}")
print(f"   Ubicada a {resultado['DISTANCIA_APROX']} km del punto de vuelo.")
print(f"   Coordenadas reales: {resultado['LATITUD']}, {resultado['LONGITUD']}")

--- 🔄 SIMULANDO PASE DE BATUTA ENTRE ETAPAS ---
📍 Dato recibido de Etapa 1: -12.02, -77.11

🎯 RESULTADO ETAPA 3:
   La estación SENAMHI más cercana es: SAN MARTIN DE PORRES
   Ubicada a 3.09 km del punto de vuelo.
   Coordenadas reales: -12.00889, -77.08447


## **Sección 3: Correlación espacial**

In [331]:
def validar_cobertura_espacial(datos_estacion):
    """
    SECCIÓN 3: AUDITORÍA DE CORRELACIÓN
    -----------------------------------------------------
    Input: Diccionario con la estación y su distancia (Output de Sección 2).
    Proceso: Evalúa si la estación está dentro de un radio útil.
    Output: Calificación de confiabilidad (Semáforo) y recomendación.
    """
    if not datos_estacion:
        return {"ESTADO": "ERROR", "MENSAJE": "No hay datos de estación para auditar."}

    distancia = datos_estacion['DISTANCIA_KM']
    estacion = datos_estacion['ESTACION']

    # --- REGLAS DE NEGOCIO (Umbrales de Distancia) ---
    # Ajustables según la sensibilidad de tu proyecto (Drones vs Aviones)
    
    if distancia <= 5.0:
        # Rango Óptimo (Barrios vecinos)
        return {
            "NIVEL_CONFIANZA": "🟢 ALTA (Excelente)",
            "REPRESENTATIVIDAD": "100% - Datos locales precisos",
            "ACCION": f"✅ Usar datos de '{estacion}' sin restricciones.",
            "RIESGO_CLIMATICO": "Bajo (Microclima idéntico)"
        }
    
    elif 5.0 < distancia <= 20.0:
        # Rango Aceptable (Misma ciudad/valle)
        return {
            "NIVEL_CONFIANZA": "🟡 MEDIA (Aceptable)",
            "REPRESENTATIVIDAD": "70% - Variación posible",
            "ACCION": f"⚠️ Usar datos de '{estacion}' con margen de error.",
            "RIESGO_CLIMATICO": "Medio (Posible variación de viento/lluvia)"
        }
    
    else:
        # Rango Crítico (Demasiado lejos)
        return {
            "NIVEL_CONFIANZA": "🔴 BAJA (No Confiable)",
            "REPRESENTATIVIDAD": "< 30% - Clima probablemente diferente",
            "ACCION": f"❌ DESCARTAR. '{estacion}' está muy lejos para validar.",
            "RIESGO_CLIMATICO": "Alto (No refleja la realidad del punto)"
        }

# ==========================================
# INTEGREMOS TODO (Simulación Final)
# ==========================================
if __name__ == "__main__":
    # 1. Simulación de lo que obtuviste en la Sección 2 (San Martín de Porres)
    resultado_seccion_2 = {
        "ESTACION": "SAN MARTIN DE PORRES",
        "LATITUD": -12.00889,
        "LONGITUD": -77.08447,
        "DISTANCIA_KM": 3.09,    # <--- Este dato es clave
        "FUENTE": "SENAMHI_MATCH"
    }

    print(f"📡 Estación candidata: {resultado_seccion_2['ESTACION']} ({resultado_seccion_2['DISTANCIA_KM']} km)")
    
    # 2. Ejecutar Sección 3
    auditoria = validar_cobertura_espacial(resultado_seccion_2)
    
    # 3. Reporte
    print("\n📋 REPORTE DE AUDITORÍA ESPACIAL:")
    print("-" * 40)
    print(f"Confianza:      {auditoria['NIVEL_CONFIANZA']}")
    print(f"Análisis:       {auditoria['REPRESENTATIVIDAD']}")
    print(f"Recomendación:  {auditoria['ACCION']}")
    print("-" * 40)

📡 Estación candidata: SAN MARTIN DE PORRES (3.09 km)

📋 REPORTE DE AUDITORÍA ESPACIAL:
----------------------------------------
Confianza:      🟢 ALTA (Excelente)
Análisis:       100% - Datos locales precisos
Recomendación:  ✅ Usar datos de 'SAN MARTIN DE PORRES' sin restricciones.
----------------------------------------


## **Sección 4: Resultados**

In [329]:
def graficar_auditoria_espacial(lat_objetivo, lon_objetivo, datos_estacion):
    """
    SECCIÓN 4: RESULTADOS VISUALES
    -----------------------------------------------------
    Genera un mapa interactivo (.html) que muestra la relación espacial
    entre el punto de interés y la estación meteorológica seleccionada.
    """
    if not datos_estacion:
        print("❌ No hay datos para graficar.")
        return

    print("🎨 Generando mapa de validación espacial...")

    # 1. Crear el mapa base centrado entre los dos puntos
    centro_lat = (lat_objetivo + datos_estacion['LATITUD']) / 2
    centro_lon = (lon_objetivo + datos_estacion['LONGITUD']) / 2
    
    m = folium.Map(location=[centro_lat, centro_lon], zoom_start=13)

    # 2. MARCADOR A: Tu Punto Objetivo (Ej. Dron/Avión)
    folium.Marker(
        location=[lat_objetivo, lon_objetivo],
        popup="📍 TU PUNTO OBJETIVO (Visual Crossing/OpenSky)",
        icon=folium.Icon(color="red", icon="plane", prefix="fa")
    ).add_to(m)

    # 3. MARCADOR B: Estación SENAMHI
    texto_estacion = f"📡 {datos_estacion['ESTACION']} ({datos_estacion['DISTANCIA_KM']} km)"
    folium.Marker(
        location=[datos_estacion['LATITUD'], datos_estacion['LONGITUD']],
        popup=texto_estacion,
        icon=folium.Icon(color="green", icon="cloud", prefix="fa")
    ).add_to(m)

    # 4. LÍNEA DE CONEXIÓN (El vector de distancia)
    folium.PolyLine(
        locations=[
            [lat_objetivo, lon_objetivo], 
            [datos_estacion['LATITUD'], datos_estacion['LONGITUD']]
        ],
        color="blue",
        weight=2,
        opacity=0.7,
        dash_array='5, 10', # Línea punteada
        tooltip=f"Distancia: {datos_estacion['DISTANCIA_KM']} km"
    ).add_to(m)

    # 5. RADIO DE CONFIANZA (Auditoría visual)
    # Dibujamos un círculo de 5km (5000 metros) alrededor de la estación
    # Si tu punto rojo cae dentro del círculo, la validación es "ALTA"
    folium.Circle(
        location=[datos_estacion['LATITUD'], datos_estacion['LONGITUD']],
        radius=5000, # 5000 metros = 5km
        color="green",
        fill=True,
        fill_opacity=0.1,
        popup="✅ Zona de Alta Confiabilidad (5km)"
    ).add_to(m)

    # 6. Guardar el resultado
    nombre_archivo = "MAPA_VALIDACION_RESULTADOS.html"
    m.save(nombre_archivo)
    print(f"✅ ¡Mapa generado! Abre el archivo '{nombre_archivo}' en tu navegador.")
    
    return m # Retorna el mapa por si estás en Jupyter Notebook para verlo ahí mismo

# ==========================================
# EJECUCIÓN FINAL (INTEGRACIÓN)
# ==========================================
if __name__ == "__main__":
    # Simulamos que ya tenemos los datos de la Sección 2
    # (En tu código real, esto viene de la variable 'mejor_estacion')
    lat_input = -12.02
    lon_input = -77.11
    
    resultado_mock = {
        "ESTACION": "SAN MARTIN DE PORRES",
        "LATITUD": -12.00889,
        "LONGITUD": -77.08447,
        "DISTANCIA_KM": 3.09
    }

    # Llamamos a la gráfica
    graficar_auditoria_espacial(lat_input, lon_input, resultado_mock)

🎨 Generando mapa de validación espacial...
✅ ¡Mapa generado! Abre el archivo 'MAPA_VALIDACION_RESULTADOS.html' en tu navegador.


# **Motor de búsqueda y enriquecimiento operativo**

## **Análisis exploratorio de datos**

Objetivo:
- Mostrar el DataFrame.head()
- Explicar qué variables se extrajeron
- info()
- describe()

In [216]:
# ===============================
# 1. OBTENER ESTACIONES EN PERÚ
# ===============================
def obtener_estaciones_peru():
    estaciones = Stations()
    estaciones = estaciones.region('PE')
    df_estaciones = estaciones.fetch()
    return df_estaciones


# ===============================
# 2. OBTENER DATOS CLIMÁTICOS
# ===============================
def obtener_datos_climaticos(lat, lon, inicio, fin):
    estaciones = Stations().nearby(lat, lon)
    estacion = estaciones.fetch(1)

    if estacion.empty:
        return pd.DataFrame()

    data = Daily(estacion.index[0], inicio, fin)
    df = data.fetch()
    return df


# ===============================
# 3. EJECUCIÓN PRINCIPAL
# ===============================
if __name__ == "__main__":

    # --- Paso 1: Estaciones ---
    print(" Obteniendo estaciones en Perú...")
    estaciones_pe = obtener_estaciones_peru()

    print("\n Vista previa de estaciones:")
    print(estaciones_pe.head())

    # --- Paso 2: Datos climáticos (ejemplo Lima) ---
    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    print("\n Descargando datos climáticos para Lima...")
    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)

    # --- Paso 3: Guardar CSV ---
    nombre_csv = "clima_lima_2024.csv"
    df_clima.to_csv(nombre_csv)
    print(f"\n Archivo CSV generado: {nombre_csv}")

    # --- Paso 4: Mostrar DataFrame ---
    print("\n Vista previa del DataFrame climático:")
    print(df_clima.head())

    print("\n Información del DataFrame:")
    print(df_clima.info())

    print("\n Estadísticas descriptivas:")
    print(df_clima.describe())


 Obteniendo estaciones en Perú...

 Vista previa de estaciones:
              name country region    wmo  icao  latitude  longitude  \
id                                                                    
84370       Tumbes      PE     TU  84370  SPME   -3.5500     -80.40   
84377      Iquitos      PE     LO  84377  SPQT   -3.7500     -73.25   
84390       Talara      PE     PI  84390  SPYL   -4.5667     -81.25   
84401        Piura      PE     PI  84401  SPUR   -5.1833     -80.60   
84405  Huancabamba      PE     PI  84405  SPAB   -5.2333     -79.45   

       elevation      timezone hourly_start hourly_end daily_start  daily_end  \
id                                                                              
84370       25.0  America/Lima   1974-03-15 2025-12-15  1974-03-15 2025-05-27   
84377      125.0  America/Lima   1973-01-01 2025-12-15  1973-01-01 2025-06-24   
84390       85.0  America/Lima   1933-02-09 2025-12-14  1942-11-01 2025-08-24   
84401       49.0  America/Lima   

Conclusión:
- El análisis exploratorio permitió validar la estructura y calidad del conjunto de datos, identificando variables relevantes y descartando aquellas con información incompleta, asegurando así una base adecuada para análisis posteriores.

## **Caracterización Climática y Generación de Indicadores Meteorológicos**

Objetivo: 
- Transformar los datos climáticos crudos del SENAMHI en indicadores meteorológicos operativos, listos para ser usados más adelante por el pipeline aéreo.

### **Selección de variables meteorológicas relevantes**

| Variable | Uso operativo                    |
| -------- | -------------------------------- |
| `wspd`   | Impacto en despegue y aterrizaje |
| `prcp`   | Riesgo de pista mojada           |
| `pres`   | Estabilidad atmosférica          |
| `tmax`   | Densidad del aire                |
| `tmin`   | Condiciones nocturnas            |

- Se seleccionaron variables meteorológicas con impacto directo en la operación aérea, descartando aquellas sin registros completos.

### **Limpieza y tratamiento de valores faltantes**

Objetivo:
- Garantizar la calidad y consistencia del conjunto de datos climático antes de su transformación en indicadores operativos.

### **Construcción de indicadores climáticos**

Objetivo:
- Transformar las variables meteorológicas continuas en indicadores que permitan detectar condiciones climáticas potencialmente adversas.
- A partir de las variables climáticas seleccionadas se construyeron indicadores binarios que permiten identificar eventos meteorológicos relevantes para la operación aérea.

Indicadores definidos:
- Viento fuerte
- Precipitación intensa
- Presión atmosférica baja

### **Clasificación de condiciones meteorológicas**

Objetivo:
- Etiquetar cada observación temporal según el nivel de severidad climática, facilitando su integración futura con datos operativos.
- Finalmente, se clasificaron las condiciones meteorológicas en categorías de severidad climática, permitiendo una interpretación directa del impacto potencial del clima.

Categorías:
- Normal
- Viento adverso
- Precipitación intensa
- Condición severa

## **Código-SENAMHI**

### **Extracción de datos**

In [233]:
def obtener_datos_climaticos(lat, lon, inicio, fin):
    estaciones = Stations().nearby(lat, lon)
    estacion = estaciones.fetch(1)

    if estacion.empty:
        return pd.DataFrame()

    data = Daily(estacion.index[0], inicio, fin)
    df = data.fetch()
    return df
print(df)

        FECHA  TMIN    PP  RESULTADO
0  2023-01-05  15.0  10.0  🌧️ LLUVIA
1  2023-02-14  12.5  25.0  🌧️ LLUVIA
2  2023-06-15  -3.0   5.0   ❄️ NIEVE
3  2023-06-20   0.5   8.0   ❄️ NIEVE
4  2023-07-01  -5.0   0.0    ☁️ SECO
5  2023-09-10   8.0   2.0  🌧️ LLUVIA
6  2023-12-24   1.8  12.0   ❄️ NIEVE


### **Limpieza de datos**

In [235]:
def limpiar_datos_climaticos(df):
    columnas_utiles = ["tavg", "tmin", "tmax", "prcp", "wspd", "pres"]
    df_limpio = df[columnas_utiles].copy()
    return df_limpio

### **Construcción de indicadores**

In [237]:
def construir_indicadores(df):
    df_ind = df.copy()

    # Rellenar NA SOLO para indicadores
    df_ind["prcp"] = df_ind["prcp"].fillna(0)
    df_ind["wspd"] = df_ind["wspd"].fillna(df_ind["wspd"].median())
    df_ind["pres"] = df_ind["pres"].fillna(df_ind["pres"].median())

    # Umbrales
    viento_umbral = df_ind["wspd"].quantile(0.90)
    lluvia_umbral = 5.0
    presion_umbral = df_ind["pres"].quantile(0.10)

    df_ind["viento_fuerte"] = (df_ind["wspd"] > viento_umbral).astype(int)
    df_ind["lluvia_intensa"] = (df_ind["prcp"] > lluvia_umbral).astype(int)
    df_ind["presion_baja"] = (df_ind["pres"] < presion_umbral).astype(int)

    return df_ind

### **Clasificación climática**

In [239]:
def clasificar_condicion(df):
    df = df.copy()
    
    score = (
        df["viento_fuerte"] +
        df["lluvia_intensa"] +
        df["presion_baja"])

    df["condicion_climatica"] = np.select(
        [score == 0, score == 1, score >= 2],
        ["Normal", "Condición moderada", "Condición severa"])
    
    return df

### **Ejecución principal + Clasificación**

In [241]:
if __name__ == "__main__":

    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)
    df_limpio = limpiar_datos_climaticos(df_clima)
    df_indicadores = construir_indicadores(df_limpio)
    df_final = clasificar_condicion(df_indicadores)

    # Guardar CSV
    df_final.to_csv("senamhi_clima_indicadores.csv")

    # VISUALIZACIÓN EN PANDAS (como pediste)
    print("\n Vista previa del dataset final:")
    print(df_final.head())

    print("\n Resumen del dataset:")
    print(df_final.info())


 Vista previa del dataset final:
            tavg  tmin  tmax  prcp  wspd    pres  viento_fuerte  \
time                                                              
2024-01-01  23.4  22.0  25.2   1.0  19.1  1014.0              0   
2024-01-02  23.8  22.0  26.0   0.0  20.9  1013.4              1   
2024-01-03  23.5  22.0  25.3   0.0  21.5  1013.5              1   
2024-01-04  23.6  21.8  26.0   0.0  16.4  1014.1              0   
2024-01-05  23.9  22.0  26.0   0.0  24.7  1013.2              1   

            lluvia_intensa  presion_baja condicion_climatica  
time                                                          
2024-01-01               0             0              Normal  
2024-01-02               0             0  Condición moderada  
2024-01-03               0             0  Condición moderada  
2024-01-04               0             0              Normal  
2024-01-05               0             0  Condición moderada  

 Resumen del dataset:
<class 'pandas.core.frame.DataFr